<h3 align="center"> CPSC 103 Project- Creating a Demographic Transition Model for Germany, China and Canada</h3> 

<img src='ProjectGraph.png'>

For my project I decided to create a Demographic Transition Model (DTM) from human geography. The focus of a DTM is to show development level in a country based on the following population indicators:

1. Crude Birth Rate 
2. Crude Death Rate
3. Total Population

The original model theorized that countries pass through 4 stages of development, however more recent versions of the model use 5 stages-  something I decided to test in my project. 

It was particularly interesting to me because just from looking at the DTM one can find areas of “natural increase” (increasing population), “natural decrease”(decreasing population), population projections based on the death and birth rates, effects of pro-natalist or anti-natalist policy changes in a country and the population momentum (as seen for China), impacts of war (Baby Boom of the 1960s in Canada), dependency ratios (Germany) etc.


Gathering data for something like might be painful - but well I literally just went to this website- 
https://ourworldindata.org/world-population-growth/#demographic-transition

Jumped to the DTM made on their site, went to their citations, and used the same csv files they used; which fortunately had the data for every country! 


So I cleaned up the csv to only have information about the country name, birth rates, death rates, total population and years. It was then defined using Compund Data and Lists like so:

<img src='ProjectGraph.png'>

In [ ]:
from typing import NamedTuple

Country_Data = NamedTuple('Country_Data', [('name',str),
                                           ('birth_rate',float),
                                          ('death_rate',float),
                                          ('year',int),
                                          ('total_population',int)])

#interp. Country Data that includes the birth rate and death rate for a particular year

CD1820= Country_Data("Germany", 39.9,24.4,1820,24905)
CD1867= Country_Data("Germany",36.8,26.1,1867,38440)
CD1917= Country_Data("Germany",13.9,20.6,1917,65763)
CD1967= Country_Data("Germany",17,11.5,1967,76832446)
CD2007= Country_Data("Germany",8.3,10.1,2007,80955210)

def fn_for_country_data(cd:Country_Data)->...:
    return ... (c.name,
                c.birth_rate,
               c.death_rate,
               c.year,
               c.total_population)




#List[Country_Data]
#interp. a list of country data for a series of years

LOCD0=[]
LOCD1= [CD1820,CD1867,CD1917]
LOCD2=[CD1820,CD1867,CD1917,CD1967,CD2007]

Note: The countries on a higher stage of the DTM (Germany at Stage 5) usually have more years of data than countries that are on a lower stage.


Which meant I had to make a choice of which time scale I should use- I could use Germany’s range as the x-axis for all the 3 countries; or use the years for each country differently. I went with the latter because the goal of a DTM is to show that some countries could start "development" soon and some later (Germany vs China) but eventually because this is a “model”, every country’s DTM is going to end up shaped roughly like Germany’s.


Additionally I specifically chose these 3 countries because they have different aspects of development (baby booms, population policies, immigration, high dependency ratio) portrayed in their DTMs.

This basically meant now I had to find some ways to add subplots and also design helpers that take in my file and give out information about one country at a time, and then implement that into a graph. A lot of the tricks I learnt were from stackoverflow forums quite frankly! And some from the matplotlib user guides.


I started with tackling the easiest- which is designing helper functions to get years, population, birth rates and death rates for each country. Here's a sample code for the helper used to get birth rates.

In [ ]:
def get_birth_rate (locd:List[Country_Data], n:str)->List[float]:
    """
    return list of birth rates
    """
    
    acc = []  #type: List[float]
    
    for cd in locd:
        if cd.name == n:
            acc.append(cd.birth_rate)
    return acc

Similar helpers were used for death_rate and population. Next I had to create my graph as subplots so I figured I would have to design them as different variables under one common figure. Here's a sample for Germany:

In [ ]:
 ax1= fig.add_subplot(212)  #212 refers to orientation
    
    years_Germany = get_year(locd,"Germany")  #more helper functions used for the axes

    ax1.set_title('Germany 1817 to 2015')     #more labels defined here
   
    ax1.plot(years_Germany, birth_rate_Germany, 'r', label = 'Birth Rate')
    ax1.plot(years_Germany, death_rate_Germany, 'g', label = 'Death Rate')
    
    #more redundant code here about the graph
    
    ax1b = ax1.twinx()
    ax1b.plot(years_Germany, total_population_Germany, 'm')
    ax1b.set_ylabel('Total Population')


So it is the same as using plt.somthing except I'd have to use ax1.something and then do that for the other two countries as well. Additonally 212 refers to the position of this plot in the entire space. Canada would be 221 (grid space of 2 vertically, 2 across and Canada is graph 1) and China would be 222 (grid space of 2 vertically, 2 across and China is graph 2). Germany would be 212 then because it would be a grid space of 2 vertically, 1 across and occupies space for 2 graphs.

ax1b refers to the total population that has a different y-scale (which I figured halfway through this project). Luckily ax1b = ax1.twinx() tells my function to plot total population on the same x-axis, but with a different y-axis on the right, and everything else would be the same.

Once I had my 3 DTMs in the same space I realized they looked extremely congested and the axes were overlapping. I thought these issues would be a quick fix but as it turns out they were the most difficult part- and took me the longest.

First I sloppily tried to write a helper that sets up tick marks on the x-axis. It worked for the most part except it does not show a label for 2015 :P


In [ ]:
def x_ticks (locd)-> float:
    """
    return the x-tick points
    """
    return (round(((max(range(len(locd)))+1)-(min(range(len(locd)))))/5))


def get_x_ticks(locd,n:str)-> List[int]:
    """
    return the x-ticks
    """
    return (np.arange(min(get_year(locd,n)), max(get_year(locd,n)),(x_ticks(get_year(locd,n)))))

##Later used as

x_ticks_Germany = get_x_ticks(locd,"Germany")
ax1.set_xticks(x_ticks_Germany) 

After which I decided to tweak the overlapping titles issue:
Seemigly menial things like removing fig.subplots_adjust(top=0.80) would mean my title runs into my graphs. 
Or the most annoying one- where I thought it said subtitle and not suptitle and spent 30 minutes trying to understand why the forums were "lying to me".

In [ ]:
    fig.suptitle('Demographic Transition Model', size=16)
    fig.tight_layout()
    fig.subplots_adjust(top=0.80)
    
    
    # \n added to fix titles from running into the right y-axis
    ax3.set_title('Canada 1900 to 2014\n')

Overall I was very content with the final result of my chart mostly because of how versatile the code could be. If I decided to add values to my file over the years or pick a different country it would generate a DTM very easily. I found it's a useful tool in human geography because just modelling something can open so much room for analysis. 

If we look at Germany - which indeed is a Stage 5 country as shown by the DTM - we can see that shortly after the 2000s, Germany's birth rate has been on a decline; the red line above the green line indicating natural decrease and the total population curve just started dipping down after 2014. So taking this a step further a population projection pyramid could be genrated or a dependency ratio analysis could be graphed.

In Canada the birth rate curve (green line) literally sees a bump between 1945 and 1969 - The post world war baby boom. With a very stable death rate and declining birth rate- Canada's total population is still increasing rapidly; an anomaly that can be explained by immigration. So an extra helper could be written that removes the effects of immigration to see if countries like Canada still follow the DTM or not, or if the DTM is an outdated way to judge to development in a country.

In China's DTM the effects of the one child policy are very pronounced on the birth rates, however the total population does not decline rapidly- this anomaly attributed to something known as population momentum. Just like momentum in physics, population momentum basically tells you a growing population will not react immediately to a negative check such as a anti-natalist policies immediately; but like in China's case it did help China dodge its tipping point in population. However, a chart can be created to analyze if a country would cross its carrying capacity and eventually hit the tipping point or not, and in the case it did, a positive check such as famine or drought would create an immediate effect in the total population.

A lot of human geography involves being able to construct models and make predictions based on current data and examples from the past, and a lot of the indicators of geography are interconnected. Like any subject a deeper analysis on the topic reveals that a lot more can be done in this field using systematic design- the above listed ideas being merely extracted from the DTM which is one single model of geography. In general I think systematic design is a pretty powerful tool in this subject area and makes data analysis very efficient.


Additonal Guide to understanding a DTM and identifying which stage a country falls under in case you are curious! :)

| Stage 1 | Stage 2 | Stage 3 | Stage 4 | Stage 5
| :------------------: | :------------------: | :------------------: | :------------------: | :------------------: 
| high birth rate | Birth rate decreases but does not drop as much; improved standard of living = bigger families | death and birth rates decrease | high levels of development |death rate exceeds birth rate (with a drop in birth rate)
|high death rate| rapid drop in death rate as diseases are cured, aid is provided, sanitation and  standards of living are improved|  increased development, more education, and negative population control checks| stable birth and death rates| high old age dependency ratios
|many diseases, high fluctuations in population, high child dependency ratios| total population begins to increase with high birth rates and low death rates| total population rises rapidly, low dependency ratios| population evens out| total population decreases
|eg: pre-industrial England, isolated tribes or war-torn regions*| eg: Malawi, Eritrea, Ethiopia|eg: India| eg: Canada| eg: Germany


**At this point in global development there is no country in Stage 1, only a few regions*